# Функторы

Функция `map` была бы логична не только для списков. Как бы она работала для Maybe?

In [1]:
f x = x + 1

x1 = Just 42
x2 = Nothing

-- хочу применить f к x1, x2, точнее, к значению внутри
-- maybeMap f x1     ->    Just 43
-- maybeMap f x2     ->    Nothing

maybeMap fun (Just n) = Just (fun n)
maybeMap _ Nothing = Nothing

print $ maybeMap f x1
print $ maybeMap f x2

Just 43

Nothing

Допустим, я хочу уметь так же применять функции ко вторым элементам пары:

In [2]:
f x = x + 1
x1 = ("hello", 42)
x2 = ([1, 2, 3], 42.0)

-- хочу применить f ко второму элементу пары

pairMap fun (x, y) = (x, fun y)

print $ pairMap f x1
print $ pairMap f x2

("hello",43)

([1,2,3],43.0)

Есть и другие аналогичные ситуации. Когда значение (или значения) хранятся в какой-то структуре с дополнительными данными.
Список - несколько значений и информация об их порядке
Maybe - одно значение или ноль
Пара - значение вместе с дополнительной информацией

Функции тоже можно понимать как такие структуры:
fun :: a -> b
получается, функция хранит значения типа b, доступ к которым осуществляется по аргументу типа a:

Например,
```
f 1 = "один"
f 2 = "два"
...
```
функция "хранит" внутри себя значения `"один"`, `"два"`, получить их можно, подставив соответствующий аргумент в функцию.

In [3]:
f x = x + 1
x1 = \x -> 2 * x
x2 = \x -> 3 * x

-- funMap f x1 - должна получиться функция, которая совпадает с x1, но после применения x1 дополнительно вызывает f

funMap fun x = \t -> fun (x t)

xx1 = funMap f x1
xx2 = funMap f x2
print $ xx1 42
print $ xx2 42

85

127

На самом деле все эти map функции уже есть. Они называются fmap или <$>, и они делают ровно то же, что делаем мы:

In [4]:
print $ fmap (+1) (Just 42)
print $ (+1) <$> Just 42

print $ (+1) <$> [10, 20, 30]
print $ (+1) <$> ("abc", 123)

Just 43

Just 43

[11,21,31]

("abc",124)

Это работает, потому что в Haskell есть класс типов Functor - функтор. Он определен так:

In [5]:
class Functor' f where
  fmap' :: (a -> b) -> f a -> f b

У него есть реализации, например, примерно так выглядит реализация для Maybe:

```
instance Functor Maybe where
  fmap f (Just x) = Just (f x)
  fmap _ Nothing = Nothing
```

Если вы определяете как функтор, вы должны гарантировать выполнение следующего правила. Функция `id` - это функция, которая ничего не делает: `id x = x`, ее тип `a -> a`.

**Правила**:
1. `fmap id x == x` (или эквивалентно) `fmap id == id`
1. `fmap (g . f) == fmap g . fmap f`

## Аппликативные функторы

Расширение обычных функторов дополнительными операциями. Представьте опять же Maybe.
Было в функторах:
1. Обычная функция
2. Данные внутри структуры
3. Применяем функцию к каждому данному внутри структуры

Теперь в аппликативных функторах можно дополнительно
1. Обычные функции внутри структуры
2. Данные внутри структуры
3. Применяем функции из структуры к данным. Как именно применять - очень сильно зависит от ситуации.

Пример с Maybe:

In [6]:
f = Just (+1) -- увеличение на 1, но внутри Maybe
x = Just 42 -- значение внутри Maybe

-- применение:
maybeApply (Just fun) (Just x) = Just (fun x)
maybeApply _ _ = Nothing

print $ maybeApply f x
print $ maybeApply Nothing x
print $ maybeApply (Just (*2)) Nothing


Just 43

Nothing

Nothing

А для списков?

In [7]:
f = [(+1), (*2)]
x = [10, 20, 30]

-- что может означать listApply, когда есть список функций и список значений?
-- Применить каждую функцию к каждому элементу, получить список новых значений.

listApply fs xs = concatMap (\f -> map f xs) fs

print $ listApply f x

[11,21,31,20,40,60]

Немного философии, почему для списков это логично делать именно так. Можно считать, что Maybe - это вычисления, в которых иногда отсутствуют результаты. А в списках можно считать, что мы делаем вычисления, в которых может быть несколько результатов.

В Haskell есть класс типов Аппликативные функторы:

In [8]:
class Functor' f => Applicative' f where
    pure :: a -> f a -- вставить значение в функтор
    (<*>) :: f (a -> b) -> f a -> f b -- та самая функция применения, рассмотренная выше
    -- и еще несколько, см. дальше

`pure` - это функция, которая помещает значение внутрь функтора, "самым естественным способом". Что это значит будет понятно из требований к аппликативному функтору, см. дальше.

Для Maybe: `pure x = Just x`, для списка `pure x = [x]`

Допустим, хотим сложить значения в двух Maybe. Или в двух списках сложить попарно значения. Как это будет написано?

In [9]:
import Control.Applicative

x1 = Just 110
x2 = Just 654

-- надо сложить. Сначала давайте получим Just (110+), потом применим ее к Just 654:
-- вынес это как отдельную задачу
fmap (110+) (Just 654)
(110+) <$> Just 654

Just (110+) <*> Just 654
Just ((+) 110) <*> Just 654

(110+) <$> [1, 2, 3]
[(110+), (220+)] <*> [1, 2, 3]

Just (110+) <*> Just 654
Just (+) <*> x1 <*> x2
(+) <$> x1 <*> x2
liftA2 (+) x1 x2
liftA2 (+) [1, 2, 3] [10, 20, 30]

Line 1: Use section
Found:
((+) 110)
Why not:
(110 +)

Just 764

Just 764

<interactive>:1:1: error:
    • Could not deduce (Applicative' Maybe) arising from a use of ‘<*>’
      from the context: Num b bound by the inferred type of it :: Num b => Maybe b at <interactive>:1:1-24
    • In the expression: Just (110 +) <*> Just 654
      In an equation for ‘it’: it = Just (110 +) <*> Just 654

Аппликативные функторы должны уметь делать не только `<*>`, но и функция `pure`, которая
вводит обычное значение в функтор.
`pure 42` в типе Maybe это `Just 42`. `pure 42` в типе списка: `[42]`.

Правила для функторов:
`pure id <*> v = v` и др по ссылке [https://hackage.haskell.org/package/base-4.12.0.0/docs/Control-Applicative.html#g:1](https://hackage.haskell.org/package/base-4.12.0.0/docs/Control-Applicative.html#g:1).

# Монады
Вычисления с Maybe - это вычисления, в которых может не получиться результа. Если мы делаем последовательность вычислений, в которой в какой-то момент результата не получилось, то и дальше считаем, что результата нет.

In [10]:
mSqrt :: Double -> Maybe Double
mSqrt x | x < 0 = Nothing
        | otherwise = Just $ sqrt x

[хотим вычислить (sqrt(x) + 1) / 2]

In [11]:
eval x = let sx = mSqrt x
             y = (+1) <$> sx
           in (/2) <$> y
eval 9
eval (-9)

Just 2.0

Nothing

In [12]:
-- sqrt(x) + sqrt(y) + 1
eval2 x y = let sx = mSqrt x
                sy = mSqrt y
                sq_plus_sq = Just (+) <*> sx <*> sy  -- liftA2 (+) sx sy
               in
                (+1) <$> sq_plus_sq

eval2 9 16
eval2 (-9) 16

<interactive>:3:30: error:
    • No instance for (Applicative' Maybe) arising from a use of ‘<*>’
    • In the expression: Just (+) <*> sx <*> sy
      In an equation for ‘sq_plus_sq’: sq_plus_sq = Just (+) <*> sx <*> sy
      In the expression:
        let
          sx = mSqrt x
          sy = mSqrt y
          sq_plus_sq = Just (+) <*> sx <*> sy
        in (+ 1) <$> sq_plus_sq

In [13]:
-- sqrt(sqrt(x) - 3)
eval3 x = let sx = mSqrt x
              sx_3 = (\t -> t - 3) <$> sx
              -- ?? не получается сделать ни с <*>, ни с <$>
              -- есть Just y и есть mSqrt :: Double -> Maybe Double
              z = case sx_3 of
                    Just n -> mSqrt n
                    Nothing -> Nothing
            in
              z
eval3 100
eval3 (-100)
eval3 4

Just 2.6457513110645907

Nothing

Nothing

Хочется вспомогательную функцию типа `<*>` и `<$>`, чтобы она имела такой заголовок:

`bind :: Maybe a -> (a -> Maybe b) -> Maybe b`.

Если подумать, то
1. `fmap` - частный случай этой функции. Сравним
   
   `bind :: Maybe a -> (a -> Maybe b) -> Maybe b`.
   
   `fmap :: Maybe a -> (a -> b) -> Maybe b`.
   
   `fmap f t = bind t (\x -> pure $ f x)` или 
   
   `fmap f t = bind t (pure . f)`
1. `<*>` - тоже частный случай. Попробуйте ее сами выразить через bind

Реализуем `bind` для Maybe:

In [14]:
maybeBind :: Maybe a -> (a -> Maybe b) -> Maybe b
maybeBind Nothing _ = Nothing
maybeBind (Just x) f = f x

-- sqrt(sqrt(x) - 3)
eval3 x = let sx = mSqrt x
              sx_3 = maybeBind sx (\t -> Just (t - 3))
             in
              maybeBind sx_3 mSqrt

eval3 100
eval3 (-100)
eval3 4

Just 2.6457513110645907

Nothing

Nothing

In [15]:
eval3 x = do
            sx <- mSqrt x
            sx_3 <- Just (sx - 3)
            mSqrt sx_3

eval3 100
eval3 (-100)
eval3 4

Just 2.6457513110645907

Nothing

Nothing

Давайте остановимся на одной вспомогательной функции

`maybeBind :: Maybe a -> (a -> Maybe b) -> Maybe b`

Функции дается значение, очередной шаг вычисления, она возвращает новое значение.

In [16]:
maybeBind :: Maybe a -> (a -> Maybe b) -> Maybe b
maybeBind Nothing _ = Nothing
maybeBind (Just x) f = f x

-- взятие корня
mSqrt :: Double -> Maybe Double
mSqrt x | x < 0 = Nothing
        | otherwise = Just $ sqrt x

-- sqrt(sqrt(x) - 3)

eval4 :: Double -> Maybe Double
-- maybeBind (Just x) mSqrt      это sqrt(x) внутри maybe
eval4 x = maybeBind
              (mSqrt x)
              (\a -> mSqrt (a - 3))

eval4 100
eval4 (-100)
eval4 3

Just 2.6457513110645907

Nothing

Nothing

Эта програма похожа на последовательность шагов:
1. `mSqrt`
2. `\a -> mSqrt (a - 3)`

Посчитаем теперь $\sqrt x + \sqrt y + 1$.

Шаги:
1. $\sqrt x$
1. $\sqrt y$
1. $\sqrt x + \sqrt y + 1$

In [17]:
eval5 :: Double -> Double -> Maybe Double
eval5 x y = maybeBind
              (mSqrt x) -- 1 шаг
              (\a -> maybeBind
                        (mSqrt y) -- 2 шаг
                        (\b -> Just (a + b + 1)) -- 3 шаг
              )

eval5 16 100
eval5 (-16) 100

Just 15.0

Nothing

`maybeBind` - это функция `>>=`, которая существует для `Maybe` и других типов:

In [18]:
eval5 :: Double -> Double -> Maybe Double
eval5 x y = (mSqrt x) >>= (\a ->
                    (mSqrt y) >>= (\b ->
                         Just (a + b + 1)
                    )
            )

eval5 16 100
eval5 (-16) 100

Just 15.0

Nothing

Для подобной записи есть do-нотация:

In [19]:
eval5 :: Double -> Double -> Maybe Double
eval5 x y = do
             a <- mSqrt x
             b <- mSqrt y
             return (a + b + 1)

eval5 16 100
eval5 (-16) 100

Just 15.0

Nothing

Получился "императивный" код. `return` для Maybe это `Just`.
Перепишем теперь `eval4`

In [20]:
-- sqrt(sqrt(x) - 3)

eval4 :: Double -> Maybe Double
--eval4 x = maybeBind
--              (mSqrt x)
--              (\a -> mSqrt (a - 3))
--
--eval4 x = (mSqrt x) >>= (\a ->
--               mSqrt (a - 3)
--          )
eval4 x = do
           a <- mSqrt x
           mSqrt (a - 3) -- сразу возвращает Maybe

--eval4 x = do                  -- эквивалентно
--           a <- mSqrt x
--           b <- mSqrt (a - 3)
--           return b

eval4 100
eval4 (-100)
eval4 3

Just 2.6457513110645907

Nothing

Nothing

Комплексное число `1 :+ 2` это $1 + 2i$.

In [21]:
import Data.Complex
lSqrt :: Complex Double -> [Complex Double]
lSqrt x = [sqrt x, -sqrt x]

lSqrt (4 :+ 0)

[2.0 :+ 0.0,(-2.0) :+ (-0.0)]

In [22]:
-- sqrt(x) + sqrt(y) + 1
eval5 :: Complex Double -> Complex Double -> [Complex Double]
eval5 x y = do
             a <- lSqrt x
             b <- lSqrt y
             return (a + b + 1)

eval5 16 100

[15.0 :+ 0.0,(-5.0) :+ 0.0,7.0 :+ 0.0,(-13.0) :+ 0.0]

Монада в хаскеле:
```
class Applicative m => Monad m where
  (>>=) :: m a -> (a -> m b) -> m b
  return :: a -> m a  -- превращает обычное значение в значение в монаде
  (>>) :: m a -> m b -> mb
  (>>) x y = x >>= (\_ -> y)
```

Пример монады. Вычисления с журналированием действий.

In [23]:
data Log a = Log a [String] -- храним значение и список сообщений

Будем пользоваться функциями, которые при вычислениях могут выдать несколько сообщений:

In [24]:
add1 :: Int -> Log Int
add1 x = Log (x + 1) ["я добавила один"]

mul2 :: Int -> Log Int
mul2 x = Log (2 * x) ["Я умножила на 2", "было тяжело"]

Скажем, что `Log` это монада и объясним, как комбинировать вычисления.

In [25]:
--instance Functor Log where
--  

--instance Applicative Log where
--  

instance Monad Log where
  --(>>=) :: Log a -> (a -> Log b) -> Log b
  (Log x messages) >>= f = let (Log y newMessages) = f x in
                            Log y (messages ++ newMessages)
  --return :: a -> Log a
  return x = Log x []

-- 2 * x + 1 через функции add1 и mul2
eval6 :: Int -> Log Int
eval6 x = do
           a <- mul2 x
           add1 a   -- или b <- add1 a; return b

eval6 10 -- должен вернуть 21 и журнал ["умножила на 2", "добавила 1"]

<interactive>:1:10: error:
    • No instance for (Applicative Log) arising from the superclasses of an instance declaration
    • In the instance declaration for ‘Monad Log’

Напишите свою реализацию